In [1]:
import obspy
import os
import glob 
import re
from obspy.io.sac import SACTrace
from obspy import read_inventory, UTCDateTime
from obspy.clients.iris import Client
from IPython.display import clear_output

In [2]:
client = Client()
root_dir = "/media/elemento/Element/NTU_RW/NEFS"
os.listdir(root_dir)

['.ipynb_checkpoints',
 '0_get_events.ipynb',
 '1_get_stations.ipynb',
 '2_plot_evtsta.ipynb',
 '3_get_waveform.ipynb',
 '3_get_waveform_2001.ipynb',
 'Download',
 'Download.png',
 'Events',
 'ISC_EHB_Catalog_1980-2018',
 'README.md',
 'Stations']

In [3]:
# Functions to read all the events one-by-one, stored in the catalog ...
# ... and store them in variables

def read_evt_NEFS(evtf):
    """Read event information"""
    with open(evtf, 'r') as f:
        lines = f.readlines()[0:]
        events = []
        dnames = []
        for line in lines:
            line = line.strip()
            temp = line.split()
            t = UTCDateTime(temp[0])
            lat, lon = float(temp[1]), float(temp[2])
            depth = float(temp[3])
            mag = float(temp[4])
            events.append([t, lat, lon, depth, mag])
            dnames.append("".join(temp[0].split("T")[0].split("-")) + "".join("".join("".join(temp[0].split("T")[1].split("Z")).split(".")).split(":")))
        return dnames, events

def read_evt_NSFE(evtf):
    """Read event information"""
    with open(evtf, 'r') as f:
        lines = f.readlines()[1:]
        events = []
        dnames = []
        for line in lines:
            line = line.strip()
            temp = line.split(',')
            t = UTCDateTime(temp[0])
            lat, lon = float(temp[1]), float(temp[2])
            depth = float(temp[3])
            mag = float(temp[5])
            events.append([t, lat, lon, depth, mag])
            dnames.append("".join(temp[0].split("T")[0].split("-")) + "".join("".join("".join(temp[0].split("T")[1].split("Z")).split(".")).split(":")))
        return dnames, events

In [4]:
def mseed2sac(data_root, data_out, data_staxml, dnames, events):
    """Convert miniseed to SAC"""
    
    # Iterating over all the events
    for evt in zip(dnames, events):
        rootd = os.path.join(data_root, evt[0])
        if not os.path.exists(rootd):
            continue
        else:
            outd = os.path.join(data_out, evt[0])
            if not os.path.exists(outd):
                os.makedirs(outd)
            
            # Moving into the folder of a single event
            # Selecting only the mseed files containing the Z component 
            fpath = os.path.join(data_root, evt[0])
            os.chdir(fpath)
            files = glob.glob("*Z.mseed")
            
            # Reading the selected files one-by-one
            for file in files:
                stanm = "{}.{}.{}.{}".format(file.split(".")[0],file.split(".")[1],file.split(".")[2],file.split(".")[3])
                if os.path.exists("{}/{}/{}.SAC".format(data_out, evt[0], stanm)):
                    continue
                else:
                    try:
                        st = obspy.read(file)
                        tr = st[0]
                        outfile = "{}.{}".format(tr.stats.network,tr.stats.station)
                        print("{} {}".format(evt[0],outfile))
                        
                        # Reading the metadata for the corresponding station ...
                        # ... of this particular data
                        stas = read_inventory(f"{data_staxml}/{outfile}.xml")

                        if (tr.stats.channel==stas[0][0][0].code):
                            stas[0][0][0].location_code = tr.stats.location
                        elif (tr.stats.channel==stas[0][0][1].code):
                            stas[0][0][1].location_code = tr.stats.location
                        else :
                            stas[0][0][2].location_code = tr.stats.location

                        # https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.remove_response.html
                        # Remove the response, and in order to do that, we need the metadata for the ...
                        # ... corresponding station
                        trace = tr.copy()
                        try:
                            trace.remove_response(inventory=stas) 
                        except ValueError:
                            print("No meaningful response File: %s" % outfile)
                            continue

                        # Convert to SACTrace from ObsPy Trace
                        sactr = SACTrace.from_obspy_trace(trace)

                        # Set event origin time as SAC reference time
                        sactr.reftime = evt[1][0]
                        sactr.o = 0
                        sactr.iztype = 'io'

                        # Set event location and magnitude
                        sactr.evla = evt[1][1]
                        sactr.evlo = evt[1][2]
                        sactr.evdp = evt[1][3]
                        sactr.mag = evt[1][4]

                        # Set station location // network, station
                        sactr.stla = stas[0][0][0].latitude
                        sactr.stlo = stas[0][0][0].longitude
                        sactr.stel = stas[0][0][0].elevation
                        sactr.knetwk = stas[0].code
                        sactr.kstnm = stas[0][0].code

                        # https://docs.obspy.org/packages/autogen/obspy.io.sac.sactrace.SACTrace.html
                        # Set cmpaz, cmpinc, kcmpnm where ...
                        # cmpaz = Component azimuth (degrees, clockwise from north)
                        # cmpinc = Component incident angle (degrees, from vertical)
                        # kcmpnm = Component name
                        if (tr.stats.channel==stas[0][0][0].code):
                            sactr.cmpaz = stas[0][0][0].azimuth
                            sactr.cmpinc = stas[0][0][0].dip
                            sactr.kcmpnm = stas[0][0][0].code
                        elif (tr.stats.channel==stas[0][0][1].code):
                            sactr.cmpaz = stas[0][0][1].azimuth
                            sactr.cmpinc = stas[0][0][1].dip
                            sactr.kcmpnm = stas[0][0][1].code
                        else :
                            sactr.cmpaz = stas[0][0][2].azimuth
                            sactr.cmpinc = stas[0][0][2].dip
                            sactr.kcmpnm = stas[0][0][2].code

                        # Other SAC headers
                        # TRUE if DIST AZ BAZ and GCARC are to be calculated from st event coordinates.
                        sactr.lcalda = 1  

                        # DIST AZ BAZ and GCARC headers will be calculated from station and event coordinates.
                        # Note that thing may not be same if we use ObsPy Trace.

                        # Write to sac files
                        sactr.write(os.path.join(outd, tr.id + '.SAC'))

                        clear_output(wait=False)
                        
                    except:
                        continue

In [5]:
providers = ["IRIS", "GFZ"]
years = [2010, 2011, 2012, 2013, 2014]

for year1 in years:
    year2 = year1 + 1
    for provider in providers:
        print(f"For the range: {year1}-{year2}, and provider: {provider}")
        
        data_mseed = f'{root_dir}/Download/{provider}/miniseed/{year1}'    
        data_staxml = f'{root_dir}/Download/{provider}/stations/{year1}'
        data_sac = f'{root_dir}/Download/{provider}/SAC/{year1}'

        ### NEFS data catalog
        evt_lst = f'{root_dir}/Events/Catalog_{year1}-{year2}'   

        ### NSFE data catalog
        # evt_lst = f'{root_dir}/events_{year1}-{year2}.csv'  

        if not os.path.exists(data_sac):
            os.makedirs(data_sac)

        # Read event metadata
        ### Using this to read catalog file when processing NEFS data
        dnames, events = read_evt_NEFS(evt_lst) 

        ### Using this to read catalog file when processing NSFE data
        # dnames, events = read_evt_NSFE(evt_lst)  

        # Convert miniseed to sac
        mseed2sac(data_mseed, data_sac, data_staxml, dnames, events)
        
        clear_output(wait=False)